In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import string
from string import digits
import re

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

from collections import Counter

In [3]:
data = pd.read_csv("/kaggle/input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv", encoding = 'utf-8')
data.head(10)

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
5,tides,The then Governor of Kashmir resisted transfer...,कश्मीर के तत्कालीन गवर्नर ने इस हस्तांतरण का व...
6,indic2012,In this lies the circumstances of people befor...,इसमें तुमसे पूर्व गुज़रे हुए लोगों के हालात हैं।
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
8,indic2012,“”Global Warming“” refer to warming caused in ...,ग्लोबल वॉर्मिंग से आशय हाल ही के दशकों में हुई...
9,tides,You may want your child to go to a school that...,हो सकता है कि आप चाहते हों कि आप का नऋर्नमेनटे...


In [4]:
data['source'].value_counts()

tides        50000
ted          39881
indic2012    37726
Name: source, dtype: int64

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127607 entries, 0 to 127606
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   source            127607 non-null  object
 1   english_sentence  127605 non-null  object
 2   hindi_sentence    127607 non-null  object
dtypes: object(3)
memory usage: 2.9+ MB


In [6]:
data=data[data['source']=='ted']

data.head(20)

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
13,ted,So there is some sort of justice,तो वहाँ न्याय है
23,ted,This changed slowly,धीरे धीरे ये सब बदला
26,ted,were being produced.,उत्पन्न नहीं कि जाती थी.
30,ted,"And you can see, this LED is going to glow.","और जैसा आप देख रहे है, ये एल.ई.डी. जल उठेगी।"
32,ted,to turn on the lights or to bring him a glass ...,"लाईट जलाने के लिए या उनके लिए पानी लाने के लिए,"
35,ted,Can you imagine saying that?,क्या आप ये कल्पना कर सकते है


In [7]:
data.isnull().sum()

source              0
english_sentence    0
hindi_sentence      0
dtype: int64

In [8]:
data.drop_duplicates(inplace = True)

In [9]:
data.describe()

,source,english_sentence,hindi_sentence
count,38803,38803,38803
unique,1,38629,38743
top,ted,(Applause),धन्यवाद |
freq,38803,39,4


In [10]:
data = data.sample(n = 30000, random_state = 42)
data.shape

(30000, 3)

In [11]:
# Lowercase all characters
data['english_sentence']=data['english_sentence'].apply(lambda x: x.lower())
data['hindi_sentence']=data['hindi_sentence'].apply(lambda x: x.lower())

In [12]:
# Define a function to apply text preprocessing steps
def preprocess_text(text):
    text = text.lower()
    text = re.sub("'", '', text)
    exclude = set(string.punctuation)
    text = ''.join(ch for ch in text if ch not in exclude)
    remove_digits = str.maketrans('', '', digits)
    text = text.translate(remove_digits)
    text = re.sub("[२३०८१५७९४६]", "", text)
    text = text.strip()
    text = re.sub(" +", " ", text)
    return text

# Apply the preprocessing function to the English and Hindi sentences
data['english_sentence'] = data['english_sentence'].apply(preprocess_text)
data['hindi_sentence'] = data['hindi_sentence'].apply(preprocess_text)
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: 'START_ ' + x + ' _END')

In [17]:
#random sample
data.sample(10)

,source,english_sentence,hindi_sentence
16725,ted,that he was enrolled in a regular school,START_ कि उसे सामान्य विद्यालय में दाखिला मिल ...
16629,ted,physical company,START_ शारीरिक साथ _END
4856,ted,mediating between wealth and power the way it ...,START_ धन और सत्ता तरह के बीच मध्यस्थता करना न...
34812,ted,no version of human morality,START_ मानवीय मूल्यों और नैतिकता के ऐसे किसी व...
64037,ted,our generation has seen both approaches tried,START_ हमारी पीढी ने दोनो ही तरह के तरीकों से ...
12617,ted,you can see a modern road hardly years old,START_ देखिये एक आधुनिक सड़क मुश्किल से साल पु...
95417,ted,in the playground or mum or father,START_ की मैदान में नकल नहीं उतारी है _END
90054,ted,we spend roughly percent of our communication ...,START_ हम लगभग अपने वार्तालाप का फीसदी हिस्सा ...
111513,ted,and its happening during our lifetimes,START_ और यह हमारे जीवन काल के दौरान हो रहा है...
13840,ted,the high line used to be covered in billboards,START_ द हाई लाइन पहले विज्ञापनों से ढकी रहती ...


In [18]:
all_eng_words = set(word for eng in data['english_sentence'] for word in eng.split())
all_hindi_words = set(word for hin in data['hindi_sentence'] for word in hin.split())

print(len(all_eng_words))
all_eng_words

15348


{'unsurpassed',
 'uppsala',
 'conversational',
 'legs',
 'canaries',
 'loan',
 'communicate',
 '“marathon”',
 'seals',
 'magazines',
 'embraced',
 'faceless',
 'commercially',
 'inbox',
 'increase',
 'rent',
 'updating',
 'stays',
 'miserable',
 'presenting',
 'errors',
 'aspen',
 'psyches',
 'dodging',
 'से',
 'theyre',
 'populates',
 'franchises',
 'scans',
 'regulate',
 'humans',
 'function',
 'porn',
 'treasure',
 'state',
 'support',
 'leaks',
 'snake',
 'thirds',
 'proper',
 'negotiated',
 'treaties',
 'caught',
 'camaguey',
 'negotiation',
 'yells',
 'rotation',
 'falcon',
 'receiving',
 'proxies',
 'in”',
 'ultrasound',
 'dim',
 'informational',
 'kilimanjaro',
 'rhetoric',
 'cartoon',
 'doubling',
 'yugoslav',
 'duterimbere',
 'patientcentered',
 'acorn',
 'polemics',
 'sprite',
 'practicalities',
 'glamorous',
 'explained',
 'flourmill',
 'vacations',
 'prices',
 'lists',
 'volatility',
 'water',
 'bigger',
 'our',
 'limb',
 'crawled',
 'dewar',
 'established',
 'engaging',
 

In [ ]:
print(len(all_hindi_words))
all_hindi_words

In [19]:
data['len_eng_sentence'] = data['english_sentence'].apply(lambda x: len(x.split(" ")))

data['len_hindi_sentence'] = data['hindi_sentence'].apply(lambda x: len(x.split(" ")))

In [20]:
data.head()

,source,english_sentence,hindi_sentence,len_eng_sentence,len_hindi_sentence
82040,ted,we still dont know who her parents are who she is,START_ हम अभी तक नहीं जानते हैं कि उसके मातापि...,11,16
85038,ted,no keyboard,START_ कोई कुंजीपटल नहीं _END,2,5
58018,ted,but as far as being a performer,START_ लेकिन एक कलाकार होने के साथ _END,7,8
74470,ted,and this particular balloon,START_ और यह खास गुब्बारा _END,4,6
122330,ted,and its not as hard as you think integrate cli...,START_ और जितना आपको लगता है यह उतना कठिन नहीं...,16,20


In [ ]:
data["eng_char_count"] = data["english_sentence"].str.len()
data["hindi_char_count"] = data["hindi_sentence"].str.len()
data["hindi_tok_count"] = data["hindi_sentence"].str.split(" ").str.len()
data["eng_tok_count"] = data["english_sentence"].str.split(" ").str.len()

In [ ]:
total_english_sentences = " "
for eng_sent in data['english_sentence'].tolist():
  total_english_sentences += eng_sent + " "

Counter(total_english_sentences.split(" ")).most_common(20)

In [ ]:
total_hindi_sentences = " "
for hindi_sent in data['hindi_sentence'].tolist():
  total_hindi_sentences += hindi_sent + " "

Counter(total_hindi_sentences.split(" ")).most_common(20)

In [ ]:
!pip install fasteda

In [ ]:
from fasteda import fast_eda

fast_eda(data[["source", "eng_char_count", "hindi_char_count", "hindi_tok_count",
               "eng_tok_count",]])

In [ ]:
data[data['len_eng_sentence']>30].shape

In [ ]:
data = data[data['len_eng_sentence']<=20]
data = data[data['len_hindi_sentence']<=20]

data

In [ ]:
print("maximum length of Hindi Sentence ",max(data['len_hindi_sentence']))
print("maximum length of English Sentence ",max(data['len_eng_sentence']))

In [ ]:
max_length_src=max(data['len_hindi_sentence'])
max_length_tar=max(data['len_eng_sentence'])

In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens, num_decoder_tokens = len(input_words), len(target_words) + 1

input_token_index = {word: i + 1 for i, word in enumerate(input_words)}
target_token_index = {word: i + 1 for i, word in enumerate(target_words)}
reverse_input_char_index = {i: word for word, i in input_token_index.items()}
reverse_target_char_index = {i: word for word, i in target_token_index.items()}

num_encoder_tokens, num_decoder_tokens

In [ ]:
data = shuffle(data)
data.head(10)

In [ ]:
X, y = data['english_sentence'], data['hindi_sentence']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,
                                                    random_state = 42)

X_train.shape, X_test.shape

In [ ]:
# Defining a batch function:
def generate_batch(X=X_train, y=y_train, batch_size=128):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),
                                          dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),
                                          dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar,
                                            num_decoder_tokens),
                                            dtype='float32')

            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size],
                                                            y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word]

                for t, word in enumerate(target_text.split()):
                    if t < len(target_text.split()) - 1:
                        decoder_input_data[i, t] = target_token_index[word]
                    if t > 0:
                        decoder_target_data[i, t - 1,
                                            target_token_index[word]] = 1.

            yield [encoder_input_data, decoder_input_data], decoder_target_data

In [ ]:
latent_dim = 300

# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [ ]:
model.summary()

In [ ]:
train_samples, val_samples = len(X_train), len(X_test)
batch_size, epochs = 128, 100

model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

In [ ]:
model.save_weights('nmt_weights.h5')

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_token_index['START_']
    decoded_sentence = ''
    
    while True:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' ' + sampled_char
        
        if (sampled_char == '_END' or len(decoded_sentence) > 50):
            break
        
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]
    
    return decoded_sentence

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

## Acknowledgements:

1. https://www.kaggle.com/code/vshantam/machine-translation-eng-hin
2. https://www.kaggle.com/code/ritesh2000/english-to-hindi-neural-machine-translation/notebook
3. https://towardsdatascience.com/english-to-hindi-neural-machine-translation-7cb3a426491f